# Script to download the 10_09_2019 folder onto a remote drive
### Making a backup in another S3 bucket did not work so this is my extra careful backup.

In [1]:
import boto3
from datetime import datetime
import os
import botocore

#### From https://stackoverflow.com/questions/31918960/boto3-to-download-all-files-from-a-s3-bucket

In [2]:
s3_resource = boto3.resource('s3')
s3_client = boto3.client('s3')

def download_dir(prefix, local, bucket, client=s3_client):
    """
    params:
    - prefix: pattern to match in s3
    - local: local path to folder in which to place files
    - bucket: s3 bucket with target contents
    - client: initialized s3 client object
    """
    keys = []
    dirs = []
    next_token = ''
    base_kwargs = {
        'Bucket':bucket,
        'Prefix':prefix,
    }
    while next_token is not None:
        kwargs = base_kwargs.copy()
        if next_token != '':
            kwargs.update({'ContinuationToken': next_token})
        results = client.list_objects_v2(**kwargs)
        contents = results.get('Contents')
        for i in contents:
            k = i.get('Key')
            if k[-1] != '/':
                keys.append(k)
            else:
                dirs.append(k)
        next_token = results.get('NextContinuationToken')
    for d in dirs:
        dest_pathname = os.path.join(local, d)
        if not os.path.exists(os.path.dirname(dest_pathname)):
            os.makedirs(os.path.dirname(dest_pathname))
    for k in keys:
        dest_pathname = os.path.join(local, k)
        if not os.path.exists(os.path.dirname(dest_pathname)):
            os.makedirs(os.path.dirname(dest_pathname))
        client.download_file(bucket, k, dest_pathname)

## This will work, I just need to pass in keys one at a time.

In [3]:
destination = "D:/S3 Bucket Backup"
BUCKET_NAME = "bucknellrobotics"
IMG_BUCKET = s3_resource.Bucket(name=BUCKET_NAME)

In [4]:
start_time = datetime.now()
i = 0
count = 0
num_already_backed_up = len(next(os.walk(destination+"/10_09_2019"))[2])

for obj in IMG_BUCKET.objects.all(): #don't want to accidentally run
    if count < num_already_backed_up: #This condition is only necessary cuz it downloaded 10,000 then messed up.
        count = count + 1
    else:
        #This should save some time by not downloading the first n
        target = obj.key
        download_dir(target, destination, BUCKET_NAME)

        i = (i + 1) % 1000 
        if i == 0:
            print(str(datetime.now()) + "   " + str(len(next(os.walk(destination+"/10_09_2019"))[2])) + " files backed up")
    if count == num_already_backed_up:
        print("Successfully skipped redownloading " + str(num_already_backed_up) + " files\n")
        count = count + 1 #Or else we would keep hitting this block
            
end_time = datetime.now()
print("DONE. Backed up " + str(len(next(os.walk(destination+"/10_09_2019"))[2])) + " files")
print("Took: " + str(end_time - start_time) + " seconds")

Successfully skipped redownloading 130540 files

2020-02-19 16:32:25.924494   131540 files backed up
2020-02-19 16:46:13.565886   132540 files backed up
2020-02-19 16:59:47.725756   133540 files backed up
2020-02-19 17:13:24.875924   134540 files backed up
2020-02-19 17:26:56.322449   135540 files backed up
2020-02-19 17:40:27.884529   136540 files backed up


KeyboardInterrupt: 